Import libraries

In [2]:
from dataset.dataset import load_data
from models import MRnet
from config import config
import torch
from torch.utils.tensorboard import SummaryWriter
from utils.utils import _train_model, _evaluate_model, _get_lr
import time
import torch.utils.data as data
import os

Method for training a model

In [1]:
"""Performs training of a specified model.
    
Input params:
    config_file: Takes in configurations to train with 
"""

def train(config : dict):
    """
    Function where actual training takes place

    Args:
        config (dict) : Configuration to train with
    """
    
    print('Starting to Train Model...')

    train_loader, val_loader, train_wts, val_wts = load_data(config['task'])

    print('Initializing Model...')
    model = MRnet()
    if torch.cuda.is_available():
        model = model.cuda()
        train_wts = train_wts.cuda()
        val_wts = val_wts.cuda()

    print('Initializing Loss Method...')
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=train_wts)
    val_criterion = torch.nn.BCEWithLogitsLoss(pos_weight=val_wts)

    if torch.cuda.is_available():
        criterion = criterion.cuda()
        val_criterion = val_criterion.cuda()

    print('Setup the Optimizer')
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, patience=3, factor=.3, threshold=1e-4, verbose=True)
    
    starting_epoch = config['starting_epoch']
    num_epochs = config['max_epoch']
    patience = config['patience']
    log_train = config['log_train']
    log_val = config['log_val']

    best_val_loss = float('inf')
    best_val_auc = float(0)

    print('Starting Training')

    writer = SummaryWriter(comment='lr={} task={}'.format(config['lr'], config['task']))
    t_start_training = time.time()

    for epoch in range(starting_epoch, num_epochs):

        current_lr = _get_lr(optimizer)
        epoch_start_time = time.time()  # timer for entire epoch

        print('Started Training')
        train_loss, train_auc = _train_model(
            model, train_loader, epoch, num_epochs, optimizer, criterion, writer, current_lr, log_every=log_train)

        print('train loop ended, now val')
        val_loss, val_auc = _evaluate_model(
            model, val_loader, val_criterion,  epoch, num_epochs, writer, current_lr, log_val)

        writer.add_scalar('Train/Avg Loss', train_loss, epoch)
        writer.add_scalar('Val/Avg Loss', val_loss, epoch)

        scheduler.step(val_loss)

        t_end = time.time()
        delta = t_end - epoch_start_time

        print("train loss : {0} | train auc {1} | val loss {2} | val auc {3} | elapsed time {4} s".format(
            train_loss, train_auc, val_loss, val_auc, delta))

        print('-' * 30)

        writer.flush()

        if val_auc > best_val_auc:
            best_val_auc = val_auc

        if bool(config['save_model']) and (epoch+1) % 10 == 0:
            file_name = 'model_{}_{}_val_auc_{:0.4f}_train_auc_{:0.4f}_epoch_{}.pth'.format(config['exp_name'], config['task'], val_auc, train_auc, epoch+1)
            torch.save({
                'model_state_dict': model.state_dict()
            }, './weights/{}/{}'.format(config['task'],file_name))

    t_end_training = time.time()
    print(f'training took {t_end_training - t_start_training} s')
    writer.flush()
    writer.close()

Train the model

In [ ]:
print('Training Configuration')
print(config)

train(config=config)

print('Training Ended...')

Training Configuration
{'max_epoch': 50, 'log_train': 100, 'lr': 1e-05, 'starting_epoch': 0, 'batch_size': 1, 'log_val': 10, 'task': 'acl', 'weight_decay': 0.01, 'patience': 5, 'save_model': 1, 'exp_name': 'test'}
Starting to Train Model...
Loading Train Dataset of acl task...
Number of -ve samples :  922
Number of +ve samples :  208
Weights for loss is :  tensor([4.4327])
Loading Validation Dataset of acl task...
Number of -ve samples :  66
Number of +ve samples :  54
Weights for loss is :  tensor([1.2222])
Initializing Model...


/home/qdy4zt/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 11 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/qdy4zt/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 11 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/qdy4zt/.local/lib/python3.11/site-packages/torchvision/models/_u

Initializing Loss Method...
Setup the Optimizer
Starting Training
Started Training


/home/qdy4zt/.local/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[Epoch: 1 / 50 | Batch : 100 / 1130 ]| Avg Train Loss 1.2117 | Train AUC : 0.4137 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 200 / 1130 ]| Avg Train Loss 1.2239 | Train AUC : 0.4217 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 300 / 1130 ]| Avg Train Loss 1.1964 | Train AUC : 0.46 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 400 / 1130 ]| Avg Train Loss 1.1516 | Train AUC : 0.4876 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 500 / 1130 ]| Avg Train Loss 1.1524 | Train AUC : 0.5156 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 600 / 1130 ]| Avg Train Loss 1.133 | Train AUC : 0.5281 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 700 / 1130 ]| Avg Train Loss 1.1355 | Train AUC : 0.552 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 800 / 1130 ]| Avg Train Loss 1.123 | Train AUC : 0.5605 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 900 / 1130 ]| Avg Train Loss 1.1194 | Train AUC : 0.576 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 1000 / 1130 ]| Avg Train Loss 1.1146 | Train AUC : 0.5861 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 1100 / 1130 ]| Avg Train Loss 

...

[Epoch: 19 / 50 | Batch : 100 / 1130 ]| Avg Train Loss 0.3045 | Train AUC : 0.9944 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 200 / 1130 ]| Avg Train Loss 0.3072 | Train AUC : 0.9895 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 300 / 1130 ]| Avg Train Loss 0.2891 | Train AUC : 0.989 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 400 / 1130 ]| Avg Train Loss 0.2864 | Train AUC : 0.9886 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 500 / 1130 ]| Avg Train Loss 0.2772 | Train AUC : 0.9889 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 600 / 1130 ]| Avg Train Loss 0.2811 | Train AUC : 0.9875 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 700 / 1130 ]| Avg Train Loss 0.2777 | Train AUC : 0.9887 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 800 / 1130 ]| Avg Train Loss 0.2789 | Train AUC : 0.9883 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 900 / 1130 ]| Avg Train Loss 0.2927 | Train AUC : 0.9861 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 1000 / 1130 ]| Avg Train Loss 0.2957 | Train AUC : 0.9859 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 1100 / 1130 ]|

/home/qdy4zt/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 11 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[Epoch: 19 / 50 | Batch : 10 / 120 ]| Avg Val Loss 0.2045 | Val AUC : 0.5 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 20 / 120 ]| Avg Val Loss 0.1238 | Val AUC : 0.5 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 30 / 120 ]| Avg Val Loss 0.0938 | Val AUC : 0.5 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 40 / 120 ]| Avg Val Loss 0.1327 | Val AUC : 0.5 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 50 / 120 ]| Avg Val Loss 0.178 | Val AUC : 0.9738 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 60 / 120 ]| Avg Val Loss 0.2602 | Val AUC : 0.9652 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 70 / 120 ]| Avg Val Loss 0.2825 | Val AUC : 0.9689 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 80 / 120 ]| Avg Val Loss 0.2648 | Val AUC : 0.9748 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 90 / 120 ]| Avg Val Loss 0.339 | Val AUC : 0.9691 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 100 / 120 ]| Avg Val Loss 0.3289 | Val AUC : 0.9651 | lr : 9e-07
[Epoch: 19 / 50 | Batch : 110 / 120 ]| Avg Val Loss 0.3211 | Val AUC : 0.965 | lr : 9e-07
train loss : 0.2978 | t

/home/qdy4zt/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 11 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[Epoch: 20 / 50 | Batch : 100 / 1130 ]| Avg Train Loss 0.2965 | Train AUC : 0.9874 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 200 / 1130 ]| Avg Train Loss 0.2605 | Train AUC : 0.9915 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 300 / 1130 ]| Avg Train Loss 0.2579 | Train AUC : 0.9928 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 400 / 1130 ]| Avg Train Loss 0.2638 | Train AUC : 0.9921 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 500 / 1130 ]| Avg Train Loss 0.2953 | Train AUC : 0.9881 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 600 / 1130 ]| Avg Train Loss 0.2943 | Train AUC : 0.9877 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 700 / 1130 ]| Avg Train Loss 0.2872 | Train AUC : 0.9879 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 800 / 1130 ]| Avg Train Loss 0.2866 | Train AUC : 0.9877 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 900 / 1130 ]| Avg Train Loss 0.2743 | Train AUC : 0.9889 | lr : 9e-07
[Epoch: 20 / 50 | Batch : 1000 / 1130 ]| Avg Train Loss 0.2726 | Train AUC : 0.9895 | lr : 9e-07
